In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score

In [2]:
#reading the csv to a pandas dataframe
user_engagement = pd.read_csv('/Users/olivianystrom/Documents/Springboard/relax_challenge/takehome_user_engagement.csv')
users = pd.read_csv('/Users/olivianystrom/Documents/Springboard/relax_challenge/takehome_users.csv', encoding = 'latin1')

In [3]:
#creating a dataframe of the adoption status of each user
user_engagement['time_stamp'] = pd.to_datetime(user_engagement['time_stamp']).dt.date
user_engagement['time_stamp'] = pd.to_datetime(user_engagement['time_stamp'])
user_engagement = user_engagement.sort_values(['user_id', 'time_stamp']).drop_duplicates()
user_engagement = user_engagement.set_index(['time_stamp', 'user_id'])

a = user_engagement.unstack().rolling('7D').sum().ge(3).any()

df = a.to_frame().reset_index()
df


,level_0,user_id,0
0,visited,1,False
1,visited,2,True
2,visited,3,False
3,visited,4,False
4,visited,5,False
...,...,...,...
8818,visited,11996,False
8819,visited,11997,False
8820,visited,11998,False
8821,visited,11999,False


In [4]:
#adding adoption status to the users dataframe
adopted_users = df[df[0] == True]['user_id'].unique().tolist()
users['adopted'] = users.apply(lambda x: int(x['object_id'] in adopted_users), axis=1)
users.dtypes

object_id                       int64
creation_time                  object
name                           object
email                          object
creation_source                object
last_session_creation_time    float64
opted_in_to_mailing_list        int64
enabled_for_marketing_drip      int64
org_id                          int64
invited_by_user_id            float64
adopted                         int64
dtype: object

In [5]:
#Replacing NaN values with 0, adding column for email provider
users['invited_by_user_id'] = users['invited_by_user_id'].fillna(0)
users['last_session_creation_time'] = users['last_session_creation_time'].fillna(0)
# users['email_provider'] = users['email'].str.split("@", 1).str[-1]
users['creation_time'] = pd.to_datetime(users['creation_time']).dt.to_period('M')
# users['creation_time'] = pd.to_datetime(users['creation_time'])
# users['creation_yearmonth'] = users['creation_time'].dt.to_period('M')

# Possible indicators: creation source, opted into mailing list, enabled for 
# marketing, invited by user 
users['invited'] = 0
users.loc[users['invited_by_user_id'] > 0, 'invited'] = 1

one_hot_encoded_data = pd.get_dummies(users, columns = ['creation_source'])
one_hot_encoded_data
X = one_hot_encoded_data[['opted_in_to_mailing_list', 
        'enabled_for_marketing_drip', 'invited', 'creation_source_GUEST_INVITE', 
        'creation_source_ORG_INVITE','creation_source_PERSONAL_PROJECTS', 'creation_source_SIGNUP',
        'creation_source_SIGNUP_GOOGLE_AUTH']]
y_ = one_hot_encoded_data[['adopted']]



In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y_, 
       test_size=0.33, random_state=42)

In [7]:
clf = RandomForestClassifier(n_estimators=300, random_state = 1,n_jobs=-1)
model_res = clf.fit(X_train, y_train)
y_pred = model_res.predict(X_test)
y_pred_prob = model_res.predict_proba(X_test)

ac = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='weighted')

print('Random Forest: Accuracy=%.3f' % (ac))

print('Random Forest: f1-score=%.3f' % (f1))

<ipython-input-7-afda2fb7d4e0>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_res = clf.fit(X_train, y_train)


Random Forest: Accuracy=0.865
Random Forest: f1-score=0.802


In [8]:
col_sorted_by_importance=clf.feature_importances_.argsort()
feat_imp=pd.DataFrame({
    'cols':X.columns[col_sorted_by_importance],
    'imps':clf.feature_importances_[col_sorted_by_importance]
})

import plotly.express as px
px.bar(feat_imp, x='cols', y='importance')

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['cols', 'imps'] but received: importance